## 配置环境

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pyhanlp import *
import numpy as np
import collections
from sys import getsizeof
import time
import re
from tqdm.notebook import tqdm_notebook as tqdm
from numba import njit # 测试

In [2]:
sighan05 = "./第二届国际中文分词评测/icwb2-data/"
SogouW = "./SogouW/Freq/SogouLabDic.dic"
msr_dict = os.path.join(sighan05, 'gold', 'msr_training_words.utf8')
# msr_test = os.path.join(sighan05, 'testing', 'msr_test.utf8') # 没用到？
msr_output = os.path.join(sighan05, 'testing', 'msr_output.txt') # 保存输出结果的空文件
msr_gold = os.path.join(sighan05, 'gold', 'msr_test_gold.utf8') # 正确结果
OUTFRMAE = "P:\t%.2f\nR:\t%.2f\nF1:\t%.2f\nOOV-R:\t%.2f\nIV-R:\t%.2f" # 准确性结果的输出格式

In [3]:
def load_dictionary(dict_file): # 加载词库
    fr = open(dict_file,encoding="utf-8")
    word_list = [item.strip().split("\t")[0] for item in fr]
    return set(word_list)

In [58]:
def to_region(segmentation: str) -> list: # 词转区间
    region = []
    start = 0
    for word in re.compile("\\s+").split(segmentation.strip()):
        end = start + len(word)
        region.append((start, end))
        start = end
    return region

def prf(gold: str, pred: str, dic) -> tuple: # 准确率函数
    A_size, B_size, A_cap_B_size, OOV, IV, OOV_R, IV_R = 0, 0, 0, 0, 0, 0, 0
    with open(gold, encoding="utf-8") as gd, open(pred, encoding="utf-8") as pd:
        for g, p in zip(gd, pd):  # 取出答案：g和预测：p
            A, B = set(to_region(g)), set(to_region(p))  # 得到区间
            A_size += len(A)
            B_size += len(B)
            A_cap_B_size += len(A & B)
            text = re.sub("\\s+", "", g)  # 得到原始文本
            for (start, end) in A:
                word = text[start:end]
                if word in dic:
                    IV += 1
                else:
                    OOV += 1

            for (start, end) in A & B:
                word = text[start:end]
                if word in dic:
                    IV_R += 1
                else:
                    OOV_R += 1
    p, r = A_cap_B_size / B_size * 100, A_cap_B_size / A_size * 100
    return p, r, 2 * p * r / (p + r), OOV_R / OOV * 100, IV_R / IV * 100

In [4]:
# 读取词典
word_dict = load_dictionary(SogouW)

## 作业

### T1
根据上面的算法流程修改前向最大匹配函数。

In [13]:
# 原函数
def forward_segment(text, dic, longest):
    word_list = []
    i = 0
    while i < len(text):
        longest_word = text[i]                      # 当前扫描位置的单字
        for j in range(i + 1, len(text) + 1):       # 所有可能的结尾
            word = text[i:j]                        # 从当前位置到结尾的连续字符串
            if word in dic:                         # 在词典中
                if len(word) > len(longest_word):   # 并且更长
                    longest_word = word             # 则更优先输出
        word_list.append(longest_word)              # 输出最长词
        i += len(longest_word)                      # 正向扫描
    return word_list

In [14]:
# 改良函数
def forward_segment_improve(text, dic, longest):
#     longest = len(max(dic,key=len)) # 得到最长单词的字数（不知道如何从前缀树中得到）
#     耗时比下面的循环久多了。。
    word_list = []
    idx = 0
    while idx < len(text):
        n = len(text) - idx # 得到当前指针pi到字串末端的字数n
        # 实现(2)
        if n == 1: 
            break
        elif n < longest:
            m = n
        else:
            m = longest
        # 实现(3)
        for section in range(m,1,-1): # 区间在缩小
            word = text[idx:idx+section]
            if word in dic:
                longest_word = word
                break
        else: # for结束时section为1，实现(b)
            longest_word = text[idx:idx+1]
            dic.add(longest_word)
        word_list.append(longest_word)
        idx += len(longest_word)
    # 实现(4)
    return word_list 

In [15]:
longest = len(max(word_dict,key=len))
print(forward_segment("江西鄱阳湖干枯，中国最大淡水湖变成大草原", word_dict, longest=longest))
print(forward_segment_improve("江西鄱阳湖干枯，中国最大淡水湖变成大草原", word_dict, longest=longest))

['江西', '鄱阳湖', '干枯', '，', '中国最大', '淡水湖', '变成', '大草原']
['江西', '鄱阳湖', '干枯', '，', '中国最大', '淡水湖', '变成', '大草原']


### T2
对比新旧算法的处理速度。  

In [18]:
def evaluate_speed_new(segment, text, dic,):
    with open(msr_gold,encoding="utf-8") as gd:
        start_time = time.time()
        alltext = 0 
        for idx,g in tqdm(enumerate(gd)): 
            if idx>=10000: # 取10000条(实际只有3984条)
                break
            text = re.sub("\\s+", "", g)
            segment(text, dic, longest)
            alltext += len(text)
        else:
            elapsed_time = time.time() - start_time
            seg_speed = alltext / elapsed_time / 10000
            print('%s: %.2f 万字/秒' % (seg.__name__, seg_speed))

In [19]:
# 仅使用一条文本进行重复发现速度浮动很大，不能正确表示速度，这里使用
text = "江西鄱阳湖干枯，中国最大淡水湖变成大草原"
segments = [forward_segment,forward_segment_improve]
res = []
for seg in segments:
    res.append((seg.__name__, evaluate_speed_new(seg, text, word_dict)))

0it [00:00, ?it/s]

forward_segment: 21.05 万字/秒


0it [00:00, ?it/s]

forward_segment_improve: 48.51 万字/秒


- 测试了几次发现浮动不大，应该是比较符合真实情况的测试。
- 这里可以发现改进后的方法确实有一定程度的提高。

### 其他一些测试

#### 字典树使用改进后的前向匹配算法的速度

In [22]:
# 字典树
class Trie:
    def __init__(self):
        self.root = {}  # 用字典构建树结构
        self.end_of_word = '#'  # 用#标志一个单词的结束

    def insert(self, word: str):
        node = self.root
        for char in word:  # 按word的每个字符搜索
            node = node.setdefault(char, {})
            # 如果找到对应字符，进入该字符的分支；如果没有则新建分支，继续生长
        node[self.end_of_word] = self.end_of_word  # 添加结束符

    # 查找一个单词是否完整的在字典树里，所以最后一句判断结束符是否在node中
    def search(self, word: str):
        node = self.root
        for char in word:
            if char not in node:
                return False
            node = node[char]  # 进入分支
        return self.end_of_word in node

    def __contains__(self, word):
        """
        实现魔术方法，可以使用in或not in判断word是否在树中
        """
        return self.search(word)

    # 查找是否有一个单词是这个前缀开始的
    def startsWith(self, prefix: str):
        node = self.root
        for char in prefix:
            if char not in node:
                return False
            node = node[char]
        return True

In [23]:
trie_tree = Trie()
for word in word_dict:
    trie_tree.insert(word)

In [24]:
# 针对字典树修改的前向匹配函数
def forward_segment_improve_tree(text, dic, longest):
#     longest = len(max(dic,key=len)) # 得到最长单词的字数（不知道如何从前缀树中得到）
#     耗时比下面的循环久多了。。
    word_list = []
    idx = 0
    while idx < len(text):
        n = len(text) - idx # 得到当前指针pi到字串末端的字数n
        # 实现(2)
        if n == 1: 
            break
        elif n < longest:
            m = n
        else:
            m = longest
        # 实现(3)
        for section in range(m,1,-1): # 区间在缩小
            word = text[idx:idx+section]
            if word in dic:
                longest_word = word
                break
        else: # for结束时section为1，实现(b)
            longest_word = text[idx:idx+1]
            dic.insert(longest_word) # 此处做了修改：插入到字典树内
        word_list.append(longest_word)
        idx += len(longest_word)
    # 实现(4)
    return word_list 

In [25]:
segments = [forward_segment,forward_segment_improve_tree]
res = []
for seg in segments:
    res.append((seg.__name__, evaluate_speed_new(seg, text, trie_tree)))

0it [00:00, ?it/s]

forward_segment: 7.88 万字/秒


0it [00:00, ?it/s]

forward_segment_improve_tree: 21.04 万字/秒


#### 准确率测试

In [69]:
runtime = time.time()-start

In [70]:
runtime

7.940832614898682

In [80]:
# forward_segment
with open(msr_gold,encoding="utf-8") as test, open(msr_output, 'w', encoding="utf-8") as output:
    start = time.time()
    for line in test:
        output.write("  ".join(forward_segment(re.sub("\\s+", "", line), word_dict, longest)))
        output.write("\n")
    runtime = time.time()-start
print(OUTFRMAE % prf(msr_gold, msr_output, word_dict))
print("Time:\t%.2f" % runtime)

P:	73.60
R:	72.75
F1:	73.17
OOV-R:	0.00
IV-R:	76.31
Time:	0.96


In [81]:
# forward_segment_improve
with open(msr_gold,encoding="utf-8") as test, open(msr_output, 'w', encoding="utf-8") as output:
    start = time.time()
    for line in test:
        output.write("  ".join(forward_segment_improve(re.sub("\\s+", "", line), word_dict, longest)))
        output.write("\n")
    runtime = time.time()-start
print(OUTFRMAE % prf(msr_gold, msr_output, word_dict))
print("Time:\t%.2f" % runtime)

P:	72.66
R:	69.23
F1:	70.90
OOV-R:	0.00
IV-R:	72.62
Time:	0.36


In [82]:
# forward_segment_improve_tree
with open(msr_gold,encoding="utf-8") as test, open(msr_output, 'w', encoding="utf-8") as output:
    start = time.time()
    for line in test:
        output.write("  ".join(forward_segment_improve_tree(re.sub("\\s+", "", line), trie_tree, longest)))
        output.write("\n")
    runtime = time.time()-start
print(OUTFRMAE % prf(msr_gold, msr_output, trie_tree))
print("Time:\t%.2f" % runtime)

P:	72.66
R:	69.23
F1:	70.90
OOV-R:	0.00
IV-R:	72.62
Time:	0.84


Q:不知道为啥，这里的测试结果和课件里的结果不一致，低好多。。然后改良的算法比原来的准确率还低一些